<a href="https://colab.research.google.com/github/ManojKumarTiwari/Pytorch/blob/master/4_MLP_Optimizing_Training_Predicting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision
from torchvision import transforms, datasets

In [0]:
import torch.nn as nn
import torch.nn.functional as F

In [0]:
import torch.optim as optim

### Fetching the data

In [4]:
train = datasets.MNIST("", train=True, download=True,
                       transform=transforms.Compose([
                                                     transforms.ToTensor()
                       ]))

test = datasets.MNIST("", train=False, download=True,
                       transform=transforms.Compose([
                                                     transforms.ToTensor()
                       ]))

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:01, 9748969.08it/s]                            


Extracting MNIST/raw/train-images-idx3-ubyte.gz to MNIST/raw


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 141952.77it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/raw


1654784it [00:00, 2330906.99it/s]                            
0it [00:00, ?it/s]

Extracting MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/raw


8192it [00:00, 53770.01it/s]            


Extracting MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/raw
Processing...
Done!


### Creating Batches of data with batch of 10 

In [0]:
trainset = torch.utils.data.DataLoader(train, batch_size=10, shuffle=True)
testset = torch.utils.data.DataLoader(test, batch_size=10, shuffle=False)

### Build the neural network architecture

In [11]:
class Net(nn.Module):

  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(28*28, 64)
    self.fc2 = nn.Linear(64, 64)
    self.fc3 = nn.Linear(64, 64)
    self.fc4 = nn.Linear(64, 10)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = self.fc4(x)

    return F.log_softmax(x, dim=1)

net = Net()
print(net)

Net(
  (fc1): Linear(in_features=784, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
)


### Using the requied loss function and optimizer according to our use case

In [0]:
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3)

### Training the model (Here we are training for 3 epochs)

In [13]:
for epochs in range(3):
  for data in trainset:
    X, y = data
    net.zero_grad()
    output = net(X.view(-1,28*28))
    loss = F.nll_loss(output, y)
    loss.backward()
    optimizer.step()
  print(loss)

tensor(0.0511, grad_fn=<NllLossBackward>)
tensor(0.4127, grad_fn=<NllLossBackward>)
tensor(0.0092, grad_fn=<NllLossBackward>)


### Evalute our model

In [15]:
correct = 0
total = 0

with torch.no_grad():
  for data in testset:
    X, y = data
    output = net(X.view(-1,28*28))
    for idx, i in enumerate(output):
      if torch.argmax(i) == y[idx]:
        correct+=1
      total+=1

print("Accuracy: ", round(correct/total))

Accuracy:  1
